In [3]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [96]:
# Load datasets into dataframes

# Basic info (gni, inflation, population) 2022, economic stability, World Bank
basic_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sub-Saharan Africa/WB_SSA.csv')

# Political stability, PRS Group, 2023
govstability_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sub-Saharan Africa/International Country Risk Guide.csv',delimiter = ';')
# Corruption Perceptions Index, Transparency International, 2022
cpi_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Sub-Saharan Africa/CPI2022_GlobalResultsTrends.xlsx',skiprows=2)

# Ranking of countries according to their quality of infrastructure in 2019, Statista, World Economic Forum
infras_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Sub-Saharan Africa/statistic_id264753_global-country-ranking-by-quality-of-infrastructure-2019.xlsx', sheet_name='Data',skiprows = 4)

# Compulsory education duration, Dec 2022, UNESCO & World Bank
edu_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sub-Saharan Africa/education.csv', skiprows = 4)


In [76]:
# Select 'Population, total', 'GNI, Atlas method (current US$)','GNI per capita, Atlas method (current US$)','Inflation, GDP deflator (annual %)', for the newest year
# higher GNI and GNI per capita - better economics
# relatively low positive inflation - stable economics
basic_2022 = basic_df[basic_df['Series Name'].isin(['Population, total', 'GNI, Atlas method (current US$)','GNI per capita, Atlas method (current US$)','Inflation, GDP deflator (annual %)'])].iloc[:,[0,2,-1]].reset_index()
#basic_2022.head()


In [113]:
basic_wide = basic_2022.pivot(index = 'Country Name', columns = 'Series Name', values = '2022 [YR2022]').reset_index()
basic_wide.columns = ['Country','GNI per capita (US$)', 'GNI (US$)', 'Inflation (annual %)','Population']
basic_wide.head()


,Country,GNI per capita (US$),GNI (US$),Inflation (annual %),Population
0,Angola,1900,67763109188.5801,16.3009255868007,35588987
1,Benin,1400,18633144077.801,4.13878000011482,13352864
2,Botswana,7350,19325490963.5174,14.5520485683065,2630296
3,Burkina Faso,840,19129288558.7701,6.05418680798384,22673762
4,Burundi,240,3037583556.66195,8.9286140055699,12889576


In [116]:
# Select to show Government Stability and only the newest month with most non-null values
# higher means more stable
govstability_2023 = govstability_df[govstability_df['Variable'] == 'Government Stability (A)'].iloc[:,[0,1,-2]]
govstable = govstability_2023.pivot(index = 'Country',columns = 'Variable',values='08/2023').reset_index()
govstable.columns = ['Country','Government Stability']
govstable.head()

,Country,Government Stability
0,Albania,7.5
1,Algeria,8.0
2,Angola,6.0
3,Argentina,5.5
4,Armenia,6.0


In [117]:
# Select to show only CPI score 2022
# higher CPI is cleaner
cpi_2022 = cpi_df[['Country / Territory','CPI score 2022']]
cpi_2022.columns = ['Country', 'CPI']
cpi_2022.head()


,Country,CPI
0,Afghanistan,24
1,Albania,36
2,Algeria,33
3,Angola,33
4,Argentina,38


In [118]:
# Top 100 countries with good infrascture in 2019, WEF
# the higher score the better
infras_2019 = infras_df.iloc[:,[1,2]]
infras_2019.columns = ['Country','Infrastructure Quality']
infras_2019.head()


,Country,Infrastructure Quality
0,Singapore,95.4
1,Netherlands,94.3
2,Hong Kong SAR,94.0
3,Switzerland,93.2
4,Japan,93.2


In [104]:
# Select newest year
edu_2022 = edu_df.iloc[:,[0,-2]]
edu_2022.columns = ['Country','Compulsory Education Duration (yrs)']
edu_2022.head()

,Country,Compulsory Education Duration (yrs)
0,Aruba,13.0
1,Africa Eastern and Southern,7.0
2,Afghanistan,9.0
3,Africa Western and Central,9.0
4,Angola,6.0


In [119]:
# Merge dataframes on the 'Country' column
merged_df = pd.merge(basic_wide, govstable, on='Country')
merged_df = pd.merge(merged_df, cpi_2022,on='Country', how = 'left')
merged_df = pd.merge(merged_df, infras_2019, on ='Country', how = 'left')
merged_df = pd.merge(merged_df, edu_2022, on='Country', how = 'left')
merged_df.head()

,Country,GNI per capita (US$),GNI (US$),Inflation (annual %),Population,Government Stability,CPI,Infrastructure Quality,Compulsory Education Duration (yrs)
0,Angola,1900,67763109188.5801,16.3009255868007,35588987,6.0,33.0,NaN,6.0
1,Botswana,7350,19325490963.5174,14.5520485683065,2630296,8.0,60.0,NaN,NaN
2,Burkina Faso,840,19129288558.7701,6.05418680798384,22673762,5.5,42.0,NaN,10.0
3,Cameroon,1660,46376088137.6987,6.24932979607975,27914536,6.5,26.0,NaN,6.0
4,Ethiopia,1020,126126312097.73,34.6840099694318,123379924,7.5,38.0,NaN,8.0


In [120]:
merged_df[merged_df['Infrastructure Quality'].notna()].head()

,Country,GNI per capita (US$),GNI (US$),Inflation (annual %),Population,Government Stability,CPI,Infrastructure Quality,Compulsory Education Duration (yrs)
15,Namibia,4880,12529991316.8044,7.21558169179639,2567012,6.5,49.0,58.5,7.0
21,South Africa,6780,406306964840.09,5.0542926833629,59893885,7.0,43.0,68.1,9.0


In [121]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Country                              28 non-null     object 
 1   GNI per capita (US$)                 28 non-null     object 
 2   GNI (US$)                            28 non-null     object 
 3   Inflation (annual %)                 28 non-null     object 
 4   Population                           28 non-null     object 
 5   Government Stability                 28 non-null     float64
 6   CPI                                  27 non-null     float64
 7   Infrastructure Quality               2 non-null      float64
 8   Compulsory Education Duration (yrs)  24 non-null     float64
dtypes: float64(4), object(5)
memory usage: 2.2+ KB


In [125]:
# Normalize the df for MCDA

# convert object to float
merged_df['GNI per capita (US$)'] = pd.to_numeric(merged_df['GNI per capita (US$)'], errors ='coerce')
merged_df['GNI (US$)'] = pd.to_numeric(merged_df['GNI (US$)'], errors ='coerce')
merged_df['Inflation (annual %)'] = pd.to_numeric(merged_df['Inflation (annual %)'], errors ='coerce')
merged_df['Population'] = pd.to_numeric(merged_df['Population'], errors ='coerce')
# columns to normalize
columns_to_norm = ['GNI (US$)','GNI per capita (US$)', 'Inflation (annual %)', 'Population', 'Government Stability', 'CPI', 'Compulsory Education Duration (yrs)']

# Min-Max scaling
for column in columns_to_norm:
  min_val = merged_df[column].min()
  max_val = merged_df[column].max()
  merged_df[column + '_normalized'] = (merged_df[column]- min_val)/(max_val - min_val)

merged_df.head()


,Country,GNI per capita (US$),GNI (US$),Inflation (annual %),Population,Government Stability,CPI,Infrastructure Quality,Compulsory Education Duration (yrs),GNI (US$)_normalized,GNI per capita (US$)_normalized,Inflation (annual %)_normalized,Population_normalized,Government Stability_normalized,CPI_normalized,Compulsory Education Duration (yrs)_normalized
0,Angola,1900,6.776311e+10,16.300926,35588987,6.0,33.0,NaN,6.0,0.141446,0.202263,0.115225,0.154704,0.333333,0.437500,0.142857
1,Botswana,7350,1.932549e+10,14.552049,2630296,8.0,60.0,NaN,NaN,0.037711,0.973126,0.105987,0.002424,1.000000,1.000000,NaN
2,Burkina Faso,840,1.912929e+10,6.054187,22673762,5.5,42.0,NaN,10.0,0.037291,0.052334,0.061097,0.095031,0.166667,0.625000,0.714286
3,Cameroon,1660,4.637609e+10,6.249330,27914536,6.5,26.0,NaN,6.0,0.095643,0.168317,0.062128,0.119245,0.500000,0.291667,0.142857
4,Ethiopia,1020,1.261263e+11,34.684010,123379924,7.5,38.0,NaN,8.0,0.266438,0.077793,0.212333,0.560325,0.833333,0.541667,0.428571


In [128]:
# assign qualitative labels for 'Infrastructure Quality' as limited data available
qual_labels = [1,0] # 1 is high, 0 is low
merged_df['Inf_qual_label'] = merged_df['Infrastructure Quality'].apply(lambda x: qual_labels[0] if x == 1 else qual_labels[1])
merged_df.head()

,Country,GNI per capita (US$),GNI (US$),Inflation (annual %),Population,Government Stability,CPI,Infrastructure Quality,Compulsory Education Duration (yrs),GNI (US$)_normalized,GNI per capita (US$)_normalized,Inflation (annual %)_normalized,Population_normalized,Government Stability_normalized,CPI_normalized,Compulsory Education Duration (yrs)_normalized,Inf_qual_label
0,Angola,1900,6.776311e+10,16.300926,35588987,6.0,33.0,NaN,6.0,0.141446,0.202263,0.115225,0.154704,0.333333,0.437500,0.142857,0
1,Botswana,7350,1.932549e+10,14.552049,2630296,8.0,60.0,NaN,NaN,0.037711,0.973126,0.105987,0.002424,1.000000,1.000000,NaN,0
2,Burkina Faso,840,1.912929e+10,6.054187,22673762,5.5,42.0,NaN,10.0,0.037291,0.052334,0.061097,0.095031,0.166667,0.625000,0.714286,0
3,Cameroon,1660,4.637609e+10,6.249330,27914536,6.5,26.0,NaN,6.0,0.095643,0.168317,0.062128,0.119245,0.500000,0.291667,0.142857,0
4,Ethiopia,1020,1.261263e+11,34.684010,123379924,7.5,38.0,NaN,8.0,0.266438,0.077793,0.212333,0.560325,0.833333,0.541667,0.428571,0


In [133]:
# Assign weights to each criterion for MCDA
weights = {
    'GNI (US$)_normalized': 0.25, #higher GNI - stronger economy and potential market demand
    'GNI per capita (US$)_normalized': 0.1, # average income per person
    'Inflation (annual %)_normalized': -0.1, # lower inflation rates - more stability
    'Government Stability_normalized': 0.25, # political stability is good for long-term investment
    'CPI_normalized':0.15, # lower corruption - more favorable for businesses
    'Inf_qual_label':0.15, # good infras (high) - better for operations
    'Compulsory Education Duration (yrs)_normalized':0.1, # more skilled workfoce
    'Population_normalized': 0.1 # larger population - larger potential workfoce, larger consumer market
}


# Calculate a suitability score for each country
merged_df['Overall_Score'] = sum(merged_df[criterion] * weights[criterion] for criterion in weights)

# Sort the data by suitability score to find the most suitable country
rank_countries = merged_df.sort_values(by='Overall_Score', ascending=False)

rank_countries[['Country','Overall_Score']]

,Country,Overall_Score
21,South Africa,0.647673
17,Nigeria,0.542710
4,Ethiopia,0.441628
23,Tanzania,0.393160
5,Gabon,0.389882
24,Togo,0.387097
18,Senegal,0.379714
15,Namibia,0.330852
6,Ghana,0.329550
25,Uganda,0.325215


In [132]:
merged_df

,Country,GNI per capita (US$),GNI (US$),Inflation (annual %),Population,Government Stability,CPI,Infrastructure Quality,Compulsory Education Duration (yrs),GNI (US$)_normalized,GNI per capita (US$)_normalized,Inflation (annual %)_normalized,Population_normalized,Government Stability_normalized,CPI_normalized,Compulsory Education Duration (yrs)_normalized,Inf_qual_label,Overall_Score
0,Angola,1900,6.776311e+10,16.300926,35588987,6.0,33.0,NaN,6.0,0.141446,0.202263,0.115225,0.154704,0.333333,0.437500,0.142857,0,0.222780
1,Botswana,7350,1.932549e+10,14.552049,2630296,8.0,60.0,NaN,NaN,0.037711,0.973126,0.105987,0.002424,1.000000,1.000000,NaN,0,NaN
2,Burkina Faso,840,1.912929e+10,6.054187,22673762,5.5,42.0,NaN,10.0,0.037291,0.052334,0.061097,0.095031,0.166667,0.625000,0.714286,0,0.224795
3,Cameroon,1660,4.637609e+10,6.249330,27914536,6.5,26.0,NaN,6.0,0.095643,0.168317,0.062128,0.119245,0.500000,0.291667,0.142857,0,0.229490
4,Ethiopia,1020,1.261263e+11,34.684010,123379924,7.5,38.0,NaN,8.0,0.266438,0.077793,0.212333,0.560325,0.833333,0.541667,0.428571,0,0.441628
5,Gabon,7540,1.800638e+10,13.778999,2388992,7.0,29.0,NaN,10.0,0.034886,1.000000,0.101903,0.001310,0.666667,0.354167,0.714286,0,0.389882
6,Ghana,2350,7.865782e+10,29.799101,33475870,6.0,43.0,NaN,11.0,0.164778,0.265912,0.186529,0.144941,0.333333,0.645833,0.857143,0,0.329550
7,Guinea,1180,1.637581e+10,12.200000,13859341,6.0,25.0,NaN,6.0,0.031394,0.100424,0.093562,0.054306,0.333333,0.270833,0.142857,0,0.152209
8,Guinea-Bissau,820,1.716919e+09,8.351497,2105566,7.5,NaN,NaN,9.0,0.000000,0.049505,0.073232,0.000000,0.833333,NaN,0.571429,0,NaN
9,Kenya,2170,1.173126e+11,6.008764,54027487,5.5,32.0,NaN,12.0,0.247562,0.240453,0.060857,0.239895,0.166667,0.416667,1.000000,0,0.308006
